In [1]:
from  lib.project_functions import * 

In [11]:
from tensorflow.keras.layers import Input, Dense, Conv2D, BatchNormalization
from tensorflow.keras.layers import Dropout, MaxPooling2D, Flatten, Activation
from tensorflow.keras import Model

from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
X, y = generator(train = True, test = True, generator_batch_size= 128)

Found 78420 images belonging to 43 classes.


In [4]:
split_ratio = 0.8
num_samples = len(X)
split_index = int(split_ratio * num_samples)

train_data = X
validation_data = train_data

# Оставьте только часть данных для валидации
train_data.samples = split_index
validation_data.samples = num_samples - split_index

In [5]:
num_classes = int(pd.read_csv("data/Train.csv", usecols= ["ClassId"]).nunique())

In [14]:
input_tensor = Input(shape=(128, 128, 3), name="input")
model1_c1 = Conv2D(32, (3, 3), name="model1_c1")(input_tensor)
bn = BatchNormalization()(model1_c1)
action = Activation('relu')(bn)

model1_c2 = Conv2D(32, (3, 3), activation='relu', name = "model1_c2")(action)

model1_c3 = Conv2D(64, (3, 3), activation='relu', name = "model1_c3")(model1_c2)
bn = BatchNormalization()(model1_c3)
action = Activation('relu')(bn)

model1_p1 = MaxPooling2D((2, 2), name = "model1_p1")(action)

model1_drop1 = Dropout(0.2)(model1_p1)
model1_flat= Flatten()(model1_drop1)

model1_d1 = Dense(128, activation='relu', name = "model1_d1")(model1_flat)
model1_d2 = Dense(256, activation='relu', name = "model1_d2")(model1_d1)
output1 = Dense(num_classes, activation='softmax', name = "outputs1")(model1_d2)
model_1 = Model(inputs=input_tensor, outputs=output1 )

model_1.compile(
    optimizer='adam', 
    loss='categorical_crossentropy',  
    metrics=['accuracy']  
)

callback = EarlyStopping(
    patience=10,
    monitor="val_loss",
    restore_best_weights=True,
)

checkpoint = ModelCheckpoint(
    filepath = " modelcheckpoint/best_model_1.h5",
    monitor='val_loss', 
    verbose=1,  
    save_best_only=True,  
    mode='min'  
)

hist = model_1.fit(train_data, epochs=100,verbose = 1, validation_data = validation_data, 
                   callbacks=[callback, checkpoint])

Epoch 1/100
  2/613 [..............................] - ETA: 1:12:27 - loss: 25.3139 - accuracy: 0.0859

KeyboardInterrupt: 

In [ ]:
model_1.save("models/model_1.h5")

In [ ]:
loaded_model = load_model("models/model_1.h5)